# Sciris demo

This file demonstrates some of the main features of Sciris. For more information, please see the tutorials:

http://docs.sciris.org/tutorials

## Getting started

First, let's make sure Sciris is installed:

In [ ]:
%pip install sciris

## Array indexing

In [ ]:
import numpy as np
import sciris as sc

minval = 0.9
data = np.random.rand(50)

In [ ]:
# Without Sciris
inds = np.nonzero(data>minval)[0]
print(f'The indices over {minval} were {", ".join([str(i) for i in inds])}.')

In [ ]:
# With Sciris
inds = sc.findinds(data>minval)
print(f'The indices over {minval} were {sc.strjoin(inds)}.')

## Powerful containers

In [ ]:
data = sc.objdict(a=[1,2,3], b=[4,5,6]) 
print(data)

In [ ]:
assert data.a == data['a'] == data[0] # Flexible options for indexing
assert data[:].sum() == 21 # You can sum a dict!
for i, key, value in data.enumitems():
  print(f'Item {i} is named "{key}" and has value {value}')

In [ ]:
import pylab as pl # Equivalent to "import matplotlib.pyplot as plt", but easier to type!
import pandas as pd

# Create some data
values = 1e6*np.random.randn(31+28)**2 # Generate some values
outliers = values > 2*values.mean() # Find outliers

In [ ]:
# Plot without Sciris
dates = pd.date_range('2022-01-01', '2022-02-28') # Create a list of dates
data = pd.DataFrame(dict(x=dates, y=values, outliers=outliers)) # Shortcut to pd.DataFrame
pl.scatter(data.x, data.y, c=data.outliers) # Vanilla Matplotlib!
pl.show()

In [ ]:
# Plot with Sciris
sc.options(jupyter=True) # Use higher-resolution plotting
dates = sc.daterange('2022-01-01', '2022-02-28', as_date=True) # Create a list of dates
data = sc.dataframe(x=dates, y=values, outliers=outliers) # Shortcut to pd.DataFrame
pl.scatter(data.x, data.y, c=data.outliers) # Vanilla Matplotlib!
sc.dateformatter() # Format a date axis nicely
sc.commaticks() # Convert the y-axis to use commas
sc.boxoff() # Turn off the box around the plot
sc.setylim() # Set y-limit (tight by default)
pl.show()

## Loading and saving

In [ ]:
class Sim:
    
    def __init__(self, days, trials):
        self.days = days
        self.trials = trials
    
    def run(self):
        self.x = np.arange(self.days)
        self.y = np.cumsum(np.random.randn(self.days, self.trials)**3, axis=0)
    
    def plot(self):
        with pl.style.context('sciris.fancy'): # Custom plot style
            pl.plot(self.x, self.y, alpha=0.6)

sim = Sim(days=100, trials=10)
sim.run()
sim.plot()

In [ ]:
# Save
sc.save('my-sim.obj', sim) # Save any Python object to disk

# Load and plot
new_sim = sc.load('my-sim.obj') # Load any Python object
new_sim.plot()

In [ ]:
# We can keep using the same object as new
new_sim.run()
new_sim.plot()

## Parallelization

In [ ]:
# Define the function to parallelize
def func(scale, x_offset, y_offset):
    np.random.seed(scale)
    data = sc.objdict() # Note the use of objdict to create a convenient container
    data.scale = scale
    data.x = x_offset+scale*np.random.randn(100)
    data.y = y_offset+scale*np.random.randn(100)
    return data

x_offset = 5
y_offset = 10
scales = [40,30,20,10] # Reverse order is easier to see when plotted

In [ ]:
# Run in parallel without Sciris
arglist = [] # Construct arguments
for scale in scales:
    args = (scale, x_offset, y_offset)
    arglist.append(args)

def helper_func(args):
    return func(*args)

import concurrent.futures
with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = executor.map(helper_func, arglist)
    results1 = list(futures)

In [ ]:
# Run in parallel with Sciris
results = sc.parallelize(func, scales, x_offset=x_offset, y_offset=y_offset)

# Plot
for data in results:
    pl.scatter(data.x, data.y, alpha=0.5, label=f'Scale {data.scale}')

## Putting it all together

Here's the "showcase" example of the wave generator from the [getting started](http://docs.sciris.org/overview.html) page, which hopefully now makes more sense!

In [ ]:
# Define random wave generator
def randwave(std, xmin=0, xmax=10, npts=50):
    np.random.seed(int(100*std)) # Ensure differences between runs
    a = np.cos(np.linspace(xmin, xmax, npts))
    b = np.random.randn(npts)
    return a + b*std

# Start timing
T = sc.timer()

# Calculate output in parallel
waves = sc.parallelize(randwave, np.linspace(0, 1, 11))

# Save to files
filenames = [sc.save(f'wave{i}.obj', wave) for i,wave in enumerate(waves)]

# Create dict from files
data = sc.odict({fname:sc.load(fname) for fname in filenames})

# Create 3D plot
sc.surf3d(data[:], cmap='orangeblue')
pl.show()

# Print elapsed time
T.toc('Congratulations, you finished the first tutorial')